In [89]:
# Get the books data from the NYPT SITE

import requests
import json
resultsList=[]
resultList=requests.get("https://api.nytimes.com/svc/books/v3/lists/names.json?api-key=8f9f412055a44f1abb9b050ccab4fa3f").json()
dir='../data/question2'
booksDirectory=dir+'/'+'Books'

#Create the Directory Structure

if not os.path.exists(booksDirectory):
    os.makedirs(booksDirectory)
    
for listName in resultList['results']:
    bookList=[]
    bookList=requests.get("https://api.nytimes.com/svc/books/v3/lists.json?list="+listName['list_name']+"&api-key=8f9f412055a44f1abb9b050ccab4fa3f").json()
    with open(resultsDir+'\\'+listName['list_name_encoded']+'.json', 'w') as outfile:
        json.dump(bookList['results'], outfile)




In [35]:
# Analysis to show how many books fall under each Category
# Analysis to Rank Books based on their rank under each category
# Take Top three Most Ranked and Least Three Ranked Books from each category

import json
import glob
import os
import csv
finalDataHash={}


bookDir='../data/question2/Books/*'
for name in glob.iglob(bookDir):
    with open(name,'r') as out:
        booksRankHash={}
        TopBottomCountHash={}

        data=json.load(out)
        for eachBook in data:
            booksRankHash[eachBook['book_details'][0]['title']]=eachBook['rank']
        sorted_book_by_rank= sorted(booksRankHash, key = lambda x: booksRankHash[x])
        top_three=sorted_book_by_rank[0:3]
        bottom_three=sorted_book_by_rank[-3:]
        TopBottomCountHash['count']=len(data)
        TopBottomCountHash['top']=top_three
        TopBottomCountHash['bottom']=bottom_three
        finalDataHash[data[1]['list_name']]=TopBottomCountHash

#Writing data to excel
        
dir=os.getcwd()
with open(dir+"/"+'booksAnalysis.csv', 'w',newline='') as csvfile:
    fieldnames = ['Category', 'No of Books','Top Ranked Books','Least Ranked Books']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for bookCategory in finalDataHash:
         writer.writerow({'Category': bookCategory,
                          'No of Books': finalDataHash[bookCategory]['count'],
                          'Top Ranked Books': finalDataHash[bookCategory]['top'][0]+','+finalDataHash[bookCategory]['top'][1]+','+finalDataHash[bookCategory]['top'][2],
                          'Least Ranked Books': finalDataHash[bookCategory]['bottom'][0]+','+finalDataHash[bookCategory]['bottom'][1]+','+finalDataHash[bookCategory]['bottom'][2]})
       


In [36]:
# Analysis to Rank publishers based on no of books Published

import json
import glob
import csv
publisherBookCount={}
for name in glob.iglob('../data/question2/Books/*'):
    with open(name,'r') as out:
        fileData=json.load(out)
        for data in fileData:
            publisher=data['book_details'][0]['publisher']
            if  publisher not in publisherBookCount:
                publisherBookCount[publisher]=1
            else:
                publisherBookCount[publisher]=publisherBookCount[publisher]+1

sorted_by_value_hashSet = sorted(publisherBookCount, key = lambda x: publisherBookCount[x], reverse = True)
ranked_dict={}
rank = 1
last_value = publisherBookCount[sorted_by_value_hashSet[0]]
ranked_dict = dict()
for name in sorted_by_value_hashSet:
    this_value = publisherBookCount[name]
    if this_value != last_value:
        rank += 1
    hash={name:publisherBookCount[name]}
    ranked_dict[str(hash)] = rank
    last_value = this_value

sorted_rank_hashSet=sorted(ranked_dict.items(), key=lambda x: x[1])
dir=os.getcwd()
with open(dir+"/"+'PublisherAnalysis.csv', 'w',newline='') as csvfile:
    fieldnames = ['Rank', 'Publisher Name','No of Publications']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for rankItem in sorted_rank_hashSet:
        words=rankItem[0].split(":")
        publisherName=words[0].translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,'./<>?\|`~-=_+"})
        count=words[1].translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,'./<>?\|`~-=_+"})
        writer.writerow({'Rank': rankItem[1], 'Publisher Name': publisherName,'No of Publications': count})


                
